BASE RELATIONNELLE

In [5]:
import pandas as pd
import numpy as np
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display
%load_ext sql
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 50
%sql sqlite:///trustscore.db




In [27]:
# Chargement des données
df_enterprises = pd.read_csv(r"C:\\Users\\MSI KATANA B13V\\Desktop\\PARCOURS DE\\Projet Fil Rouge\\de_satisfaction_client\\data\\informations_entreprises.csv", sep=',')
df_comments = pd.read_json(r"C:\\Users\\MSI KATANA B13V\\Desktop\\PARCOURS DE\\Projet Fil Rouge\\de_satisfaction_client\\data\\commentaires.json")

  

In [45]:
import pandas as pd
import sqlite3

def insert_companies_and_reviews(df):
    """
    Insère les données d'entreprises et de reviews dans les tables appropriées.
    
    Paramètres:
    df (DataFrame): Le DataFrame contenant les données des entreprises et des reviews.
    """
    # Connexion à la base de données SQLite
    conn = sqlite3.connect('trustscore.db')
    
    # Préparation et insertion des données des entreprises
    df['five_star_percentage'] = df['five_star_%'].str.rstrip('%').astype(float) / 100
    df_companies = df[['company_name', 'town', 'country', 'institution_type']].copy()  # Crée une copie explicite
    df_companies.to_sql('companies', conn, if_exists='append', index=False, method='multi')
    
    # Récupération des IDs des entreprises insérées
    ids = pd.read_sql_query("SELECT id FROM companies", conn)
    df_companies['id'] = ids['id']  # Affectation directe sur une copie

    # Préparation des données des évaluations
    df_reviews = df.merge(df_companies, on='company_name', how='left')
    df_reviews = df_reviews[['id', 'review', 'trust_score', 'five_star_percentage']]
    df_reviews.columns = ['company_id', 'review', 'trust_score', 'five_star_percentage']

    # Insertion des données dans la table `reviews`
    df_reviews.to_sql('reviews', conn, if_exists='append', index=False, method='multi')

    print(f"Nombre de lignes dans le CSV initial : {len(df_enterprises)}")
    print(f"Nombre de lignes après traitement : {len(df_companies)}")
    print(f"Nombre de lignes insérées dans la base de données : {pd.read_sql_query('SELECT COUNT(*) FROM companies', conn).iloc[0, 0]}")

    # Fermer la connexion
    conn.close()


In [46]:
# Assurez-vous que df contient toutes les colonnes nécessaires
insert_companies_and_reviews(df_enterprises)


Nombre de lignes dans le CSV initial : 20
Nombre de lignes après traitement : 20
Nombre de lignes insérées dans la base de données : 424


count(*)
120


In [44]:
%%sql
select count(*) from companies;

count(*)
404


BASE DE DONNEE ORIENTE OBJET

In [47]:
from pymongo import MongoClient
import json

def insert_data_from_json_to_mongodb(json_file_path, db_name='trustscore', collection_name='commentaires', mongo_uri='mongodb://localhost:27017/'):
    """
    Lit les données d'un fichier JSON et les insère dans une collection MongoDB.

    Paramètres:
    - json_file_path (str): Le chemin du fichier JSON contenant les données.
    - db_name (str): Le nom de la base de données.
    - collection_name (str): Le nom de la collection.
    - mongo_uri (str): L'URI de connexion à MongoDB.

    Retourne:
    - result (InsertManyResult): Le résultat de l'insertion.
    """
    try:
        # Connexion à MongoDB
        client = MongoClient(mongo_uri)

        # Sélection de la base de données et de la collection
        db = client[db_name]
        collection = db[collection_name]

        # Lecture des données du fichier JSON
        with open(json_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # Insertion des données
        if isinstance(data, list):
            # Insérer plusieurs documents
            result = collection.insert_many(data)
            print(f"{len(result.inserted_ids)} documents ont été insérés.")
        else:
            # Insérer un seul document
            result = collection.insert_one(data)
            print(f"Document inséré avec l'ID : {result.inserted_id}")

        return result

    except Exception as e:
        print(f"Erreur lors de l'insertion : {e}")

    finally:
        # Fermer la connexion
        client.close()




In [48]:
# Utilisation de la fonction pour insérer les données du fichier JSON
json_file_path = r'C:\Users\MSI KATANA B13V\Downloads\de_satisfaction_client-develop\de_satisfaction_client-develop\df_commentaires_par_entreprise.json'  # Remplacez par le chemin de votre fichier JSON
insert_data_from_json_to_mongodb(json_file_path)

184 documents ont été insérés.


InsertManyResult([ObjectId('66d09e5ced56c618fb099b28'), ObjectId('66d09e5ced56c618fb099b29'), ObjectId('66d09e5ced56c618fb099b2a'), ObjectId('66d09e5ced56c618fb099b2b'), ObjectId('66d09e5ced56c618fb099b2c'), ObjectId('66d09e5ced56c618fb099b2d'), ObjectId('66d09e5ced56c618fb099b2e'), ObjectId('66d09e5ced56c618fb099b2f'), ObjectId('66d09e5ced56c618fb099b30'), ObjectId('66d09e5ced56c618fb099b31'), ObjectId('66d09e5ced56c618fb099b32'), ObjectId('66d09e5ced56c618fb099b33'), ObjectId('66d09e5ced56c618fb099b34'), ObjectId('66d09e5ced56c618fb099b35'), ObjectId('66d09e5ced56c618fb099b36'), ObjectId('66d09e5ced56c618fb099b37'), ObjectId('66d09e5ced56c618fb099b38'), ObjectId('66d09e5ced56c618fb099b39'), ObjectId('66d09e5ced56c618fb099b3a'), ObjectId('66d09e5ced56c618fb099b3b'), ObjectId('66d09e5ced56c618fb099b3c'), ObjectId('66d09e5ced56c618fb099b3d'), ObjectId('66d09e5ced56c618fb099b3e'), ObjectId('66d09e5ced56c618fb099b3f'), ObjectId('66d09e5ced56c618fb099b40'), ObjectId('66d09e5ced56c618fb099b